# Transfer Learning - I

This notebook trains data from the HAM 10000 skin lesion dataset on a simple U-Net styled model. The dataset is taken from a [Kaggle Kernel](https://www.kaggle.com/alexako/cs200-1-u-net-skin-lesion-segmentation) while all processing has been done on Google Colab.

## Imports

In [ ]:
# Utils Imports

import cv2
import numpy as np
import glob
import os
from pathlib import Path
from PIL import Image
import random

In [ ]:
# Keras Imports

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Activation, Dense, BatchNormalization, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.optimizers import adam_v2 #, SGD


In [ ]:
# Notebook Support

from sklearn.model_selection import train_test_split
from warnings import filterwarnings

# import numpy as np
# import pandas as pd
# import glob
# import PIL
# from PIL import Image
# import matplotlib.pyplot as plt
# import cv2t
# %matplotlib inline

## Colab Utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
root_folder = r"/content/drive/MyDrive/Spring 2022/Machine Learning for Physical Applications/Project/Manas Trials/Self-Supervised/"

## Generate Training Data

Images of skin and their corresponding lesion masks are taken from the dataset and loaded. This data is further used to split and create training and testing pools.

In [ ]:
# Load Data

filelist_trainx = sorted(glob.glob(os.path.join(root_folder, "archive/ph2_resized", "trainx","*.bmp"))) #, key=numericalSort
X_train = np.array([np.array(Image.open(fname)) for fname in filelist_trainx])

filelist_trainy = sorted(glob.glob(os.path.join(root_folder, "archive/ph2_resized", "trainy","*.bmp"))) # , key=numericalSort
Y_train = np.array([np.array(Image.open(fname)) for fname in filelist_trainy])

print(filelist_trainx[21])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size = 0.25, random_state = 101)

## Define Metrics

Using Keras backend to make metrics for Intersection-over-Union, Dice, Precision, Recall and Accuracy

In [ ]:
# glob.glob(os.path.join(root_folder, "archive/ph2_resized", "trainx","*.bmp"))
def iou(y_true, y_pred, smooth = 100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.square(y_true), axis = -1) + K.sum(K.square(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

In [ ]:
def dice_coef(y_true, y_pred, smooth = 100):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
def precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
def recall(y_true, y_pred):
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
def accuracy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    return K.mean(K.equal(y_true, K.round(y_pred)))

## Online Data Augmentation

Data is augmented by rotation and flip at random angles and directions.

In [ ]:
# Random Rotation

def random_rotation(x_image, y_image):
    rows_x,cols_x, chl_x = x_image.shape
    rows_y,cols_y = y_image.shape
    rand_num = np.random.randint(-40,40)
    M1 = cv2.getRotationMatrix2D((cols_x/2,rows_x/2),rand_num,1)
    M2 = cv2.getRotationMatrix2D((cols_y/2,rows_y/2),rand_num,1)
    x_image = cv2.warpAffine(x_image,M1,(cols_x,rows_x))
    y_image = cv2.warpAffine(y_image.astype('float32'),M2,(cols_y,rows_y))
    return x_image, y_image.astype('int')

In [ ]:
# Flip Augmentation

def horizontal_flip(x_image, y_image):
    x_image = cv2.flip(x_image, 1)
    y_image = cv2.flip(y_image.astype('float32'), 1)
    return x_image, y_image.astype('int')

In [ ]:
# Combined Augmentation Function

def img_augmentation(x_train, y_train):
    x_rotat = []
    y_rotat = []
    x_flip = []
    y_flip = []
    x_nois = []
    for idx in range(len(x_train)):
        x,y = random_rotation(x_train[idx], y_train[idx])
        x_rotat.append(x)
        y_rotat.append(y)
        
        x,y = horizontal_flip(x_train[idx], y_train[idx])
        x_flip.append(x)
        y_flip.append(y)
        return np.array(x_rotat), np.array(y_rotat), np.array(x_flip), np.array(y_flip)

In [ ]:
# Augmentation on data

x_rotated, y_rotated, x_flipped, y_flipped = img_augmentation(x_train, y_train)
x_rotated_t, y_rotated_t, x_flipped_t, y_flipped_t = img_augmentation(x_test, y_test)

In [ ]:
# Generating overall augmented dataset

x_train_full = np.concatenate([x_train, x_rotated, x_flipped])
y_train_full = np.concatenate([y_train, y_rotated, y_flipped])

In [ ]:
# Clear not needed variables from RAM

del x_train
del x_rotated
del x_flipped
del y_train
del y_rotated
del y_flipped

### Final Train Test Split for model

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size = 0.30, random_state = 101)

In [ ]:
print("Length of the Training Set   : {}".format(len(x_train)))
print("Length of the Test Set       : {}".format(len(x_test)))
print("Length of the Validation Set : {}".format(len(x_val)))

## Model for Lesion Detection

The model uses and encoder-bottleneck-decoder approach towards detecting lesions on the skin. Encoder and Decoder are defined separately.

In [ ]:
def encoder(img_input):
    x = Conv2D(64, (3, 3), padding='same', name='conv1',strides= (1,1))(img_input)
    x = BatchNormalization(name='bn1')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(128, (3, 3), padding='same', name='conv3')(x)
    x = BatchNormalization(name='bn3')(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same', name='conv4')(x)
    x = BatchNormalization(name='bn4')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(256, (3, 3), padding='same', name='conv5')(x)
    x = BatchNormalization(name='bn5')(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv6')(x)
    x = BatchNormalization(name='bn6')(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv7')(x)
    x = BatchNormalization(name='bn7')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(512, (3, 3), padding='same', name='conv8')(x)
    x = BatchNormalization(name='bn8')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv9')(x)
    x = BatchNormalization(name='bn9')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv10')(x)
    x = BatchNormalization(name='bn10')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(512, (3, 3), padding='same', name='conv11')(x)
    x = BatchNormalization(name='bn11')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv12')(x)
    x = BatchNormalization(name='bn12')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv13')(x)
    x = BatchNormalization(name='bn13')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Dense(1024, activation = 'relu', name='fc1')(x)
    x = Dense(1024, activation = 'relu', name='fc2')(x)

    return x

In [ ]:
def decoder(enc):
    # Decoding Layer 
    x = UpSampling2D()(enc)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv1')(x)
    x = BatchNormalization(name='bn14')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv2')(x)
    x = BatchNormalization(name='bn15')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv3')(x)
    x = BatchNormalization(name='bn16')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv4')(x)
    x = BatchNormalization(name='bn17')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv5')(x)
    x = BatchNormalization(name='bn18')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv6')(x)
    x = BatchNormalization(name='bn19')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv7')(x)
    x = BatchNormalization(name='bn20')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv8')(x)
    x = BatchNormalization(name='bn21')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv9')(x)
    x = BatchNormalization(name='bn22')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv10')(x)
    x = BatchNormalization(name='bn23')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv11')(x)
    x = BatchNormalization(name='bn24')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv12')(x)
    x = BatchNormalization(name='bn25')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv13')(x)
    x = BatchNormalization(name='bn26')(x)
    x = Activation('sigmoid')(x)
    return x


In [ ]:
def unet():
    # Input layer
    img_input = Input(shape= (None, None, 3))
    
    # Encoder bottleneck
    enc = encoder(img_input)
    
    # Decoder upsampling
    dec = decoder(enc)
    
    # Resampling prediction for data
    pred = Reshape((192,256))(dec)
    
    # Create collective model
    model = Model(inputs=img_input, outputs=pred)
    
    # Compile model
    model.compile(optimizer= adam_v2.Adam(learning_rate=0.001), loss= ["binary_crossentropy"]
                  , metrics=[iou, dice_coef, precision, recall, accuracy])
    # , momentum=0.9, decay=0.0005, nesterov=False)
    print("Model compiled successfully") #model.summary()
    return model

## Training model

In [ ]:
# Training function that takes epochs and saves model weights

def train(epochs_num, savename):
    model = unet() #epochs_num, savename)
    hist = model.fit(x_train, y_train, epochs= epochs_num, batch_size= 18, validation_data= (x_val, y_val), verbose=1)
    model.save(savename)
    return model,hist

In [ ]:
# Define hyperparameters

base_epochs = 100
weight_path = os.path.join(root_folder,'unet_'+str(base_epochs)+'_epoch.h5')

In [ ]:
# Final training

model, hist = train(base_epochs, weight_path)

## Validation

We take a sample image and run entire pipeline to get a sanity check on the model and its effectiveness

In [ ]:
img = tf.convert_to_tensor(np.expand_dims(x_train[21],0), dtype='float')

In [ ]:
encoded = encoder(img)
print(encoded.shape)

### Create model and load weights

In [ ]:
model0 = unet()

In [ ]:
model0.load_weights(weight_path)

In [ ]:
model0.summary()

### Setup model details upto 

In [ ]:
out = model0.predict(img)
print(out)